In [4]:
import pandas as pd

def was_channel_jammed(channel, eventDf):
    filtered_df = eventDf[(eventDf['shortChannelId_outgoing'] == channel) | 
                          (eventDf['shortChannelId_incoming'] == channel)]
    slot_count = 0
    time_slot_list = []
    for index, row in filtered_df.iterrows():
        if row['eventType'] == 'add':
            slot_count += 1
            time_slot_list.append({'time': row['eventTimeNs'], 'taken_slots': slot_count})
        elif row['eventType'] == 'resolve':
            slot_count -= 1
            time_slot_list.append({'time': row['eventTimeNs'], 'taken_slots': slot_count})
    result_df = pd.DataFrame(time_slot_list)
    #result_df = result_df.sort_values(by='time').reset_index(drop=True)
    return result_df

def process_all_channels(pair_schedule_df):
    unique_channels = pd.unique(
        pair_schedule_df[['shortChannelId_incoming', 'shortChannelId_outgoing']].values.ravel('K')
    )
    channel_results = {}
    for channel in unique_channels:
        result_df = was_channel_jammed(channel, pair_schedule_df)
        channel_results[channel] = result_df
    return channel_results

def find_channels_with_high_slots(all_channel_results, threshold=50):
    channels_with_high_slots = []
    for channel, df in all_channel_results.items():
        if (df['taken_slots'] > threshold).any():
            channels_with_high_slots.append(channel)
    return channels_with_high_slots



In [197]:
def calculate_cumulative_time(chan_df, from_slots):
    cumulative_time_ns = 0
    start_time = None

    for i in range(len(chan_df)):
        if chan_df.iloc[i]['taken_slots'] >= from_slots and start_time is None:
            start_time = chan_df.iloc[i]['time']
        elif chan_df.iloc[i]['taken_slots'] < from_slots and start_time is not None:
            cumulative_time_ns += chan_df.iloc[i]['time'] - start_time
            start_time = None

    # Convert time from nanoseconds to minutes
    cumulative_time_minutes = cumulative_time_ns / (1e9 * 60)
    return cumulative_time_minutes

In [198]:
import pandas as pd
import json

def create_pair_schedule_df(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    
    forwards = data.get('forwards', [])
    
    # Extract relevant fields
    rows = []
    for entry in forwards:
        if isinstance(entry, dict):
            rows.append({
                'eventTimeNs': int(entry['addTimeNs']),
                'eventType': 'add',
                'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
            })
            if 'resolveTimeNs' in entry:
                rows.append({
                    'eventTimeNs': int(entry['resolveTimeNs']),
                    'eventType': 'resolve',
                    'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                    'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
                })
        else:
            print(f"Expected a dictionary but got {type(entry)}.")
    
    # Create DataFrame
    pair_schedule_df = pd.DataFrame(rows)
    pair_schedule_df = pair_schedule_df.sort_values(by='eventTimeNs').reset_index(drop=True)
    print(pair_schedule_df.head())
    return pair_schedule_df

FolderNumber = 16
json_file_path = f'JamAttackEval/results_Jul23/slowjam/{FolderNumber}/forwarding_history.json'
pair_schedule_df = create_pair_schedule_df(json_file_path)

# Now you can process the DataFrame as required
all_channel_results = process_all_channels(pair_schedule_df)
channels_with_high_slots = find_channels_with_high_slots(all_channel_results, 370)

print("Channels with taken slots above x:", channels_with_high_slots)

           eventTimeNs eventType  shortChannelId_outgoing   
0  1721217612303718184       add          393625162809344  \
1  1721217617509669184   resolve          393625162809344   
2  1721217637005228184       add          393625162809344   
3  1721217639353324184   resolve          393625162809344   
4  1721217666948642184       add          393625162809344   

   shortChannelId_incoming  
0          378232000020480  
1          378232000020480  
2          378232000020480  
3          378232000020480  
4          378232000020480  
Channels with taken slots above x: [393625162809344]


In [200]:
if pair_schedule_df is not None:
    all_channel_results = process_all_channels(pair_schedule_df)
    channels_with_high_slots = find_channels_with_high_slots(all_channel_results, 482)

    print("Channels with taken slots above x:", channels_with_high_slots)

    # Assuming there is only one channel with high slots, get the first one
    if channels_with_high_slots:
        selected_channel = channels_with_high_slots[0]
        chan_df = was_channel_jammed(selected_channel, pair_schedule_df)
        
        #max_slot = 483
        min_for_jam = 480
        
        cumulative_time = calculate_cumulative_time(chan_df, min_for_jam)
        print(f"In Folder {FolderNumber}, cumulative time where taken slots are above {min_for_jam}: {cumulative_time}")
    else:
        print(f"Folder {FolderNumber} has no jams with slot min {min_for_jam}")
else:
    print("Failed to create pair schedule DataFrame.")

Channels with taken slots above x: [393625162809344]
In Folder 16, cumulative time where taken slots are above 480: 10.063066915033334


Channels with taken slots above x: [393625162809344]

Cumulative time where taken slots changed from 483 to 482: 0.00019307861666666666

474-471

Folder 2 -> 1.1512909235333333
Folder 3 -> 0.00019307861666666666
Folder 6 -> 0.00019307861666666666

Folder 1 -> 10.142129792066667
Folder 4 -> 9.90197457735
Folder 5 -> 10.048196056033333
Folder 7 -> 10.10537017445
Folder 8 -> 10.039394890083333
Folder 9 -> 9.801070407716667

Prob Folder 9

# OLD

In [165]:
chan_df = was_channel_jammed(393625162809344, pair_schedule_df)

In [168]:
chan_df[chan_df['taken_slots']>=480]

,time,taken_slots
65257,1721308107670202543,480
65739,1721308154207257624,480
65740,1721308154372220022,481
65741,1721308154381065281,482
65742,1721308154393579407,483
65743,1721308169997600279,482
65744,1721308170703955473,481
65745,1721308171319383542,480
66221,1721308199571599196,480
66222,1721308199766490794,481


In [148]:
chan_df['taken_slots'].max()

453

In [150]:
1721399528932451460 - 1721400205875257642

-676942806182

In [155]:
1721327094592967554 - 1721327773175903274

-678582935720

In [160]:
1721323921847935496 - 1721324133441487272

-211593551776

In [161]:
ls

BData/                       feeEst/
Data Study/                  jam slot count.ipynb
JamAttackEval/               ln-jamming-simulator/
LNStat/                      ln-jamming-simulator 15 Aug/
Maypoles/                    ln-jamming-simulator.zip
bitcoin-data-analysis/       topology/
blockbuilding/               wip-lightning-jamming-paper/
fee-estimates-analysis/
